# Colab Imports

In [1]:
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

Mounted at /content/drive


## Installation

In [ ]:
!pip install -qU langchain-community faiss-cpu langchain-openai

## Env Setting

In [3]:
import os

os.environ['OPENAI_API_KEY'] = userdata.get('openAI')

In [4]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [5]:
# Index Path(Directory to save initialized index)
DRIVE_PATH = '/content/drive/MyDrive/RAG_JSON_EMBEDDINGS_INDEX'
INDEX_DIR_PATH = os.path.join(DRIVE_PATH, "INDEX")
HA_INDEX_PATH = os.path.join(INDEX_DIR_PATH, "HA_INDEX")
if not os.path.exists(INDEX_DIR_PATH):
    os.makedirs(INDEX_DIR_PATH)

faiss_index_path = os.path.join(HA_INDEX_PATH, "combined_faiss_index")

print(faiss_index_path)

/content/drive/MyDrive/RAG_JSON_EMBEDDINGS_INDEX/INDEX/HA_INDEX/combined_faiss_index


## Index Loading

In [7]:
# Faiss index set
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

loaded_vector_store = FAISS.load_local(
    faiss_index_path, embeddings, allow_dangerous_deserialization=True
    )
faiss_retriever = loaded_vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5})

### Test

In [11]:
results = faiss_retriever.invoke("성북구 고기 맛집")

In [14]:
for row in results:
  print(row.page_content)

성북구 [SEP] 삼선동 [SEP] 서울 성북구 동소문로6길 3-4 [SEP] 맛있으면 돼지 [SEP] None [SEP] 별점 4.49 [SEP] https://naver.me/FIgYRlhY [SEP] 벼르다가 다녀왔네요~ 친절하십니다~ 고기가너무맛있어요 기본나오는 콩나물국은 얼큰해서 소주한잔하기 너무좋아요 친절하신 사장님두분😁 추천받아 간 집이었는데 고기 맛도 좋고 사장님께서도 친절하셨어요 . 찍어먹는 소금 , 색도 예쁘고 눈으로도 잘 먹었어요 🎶 후회가 없는 고기집 맛나고 좋아요 이근처에 맛있는 삼겹살집이 없어서 늘 아쉬웠는데 드디어 찾았네요..매장이 엄청 깔끔하고 고기가 진짜 맛있어요.. 깔끔하고 담백한 고깃집입니다. 와보시면 깔끔하고 담백하다는말이 무슨뜻인지 알거에요 ㅎㅎ [SEP] 소개 '맛돼지'는 100%국내산 생고기만을 취급하며 고기를 두툼하게 썰어 육즙을 최대한 느끼실 수 있으며 질 좋은 숯을 사용하여 고기의 풍미를 살렸으며 함초소금, 생와사비, 갈치속젓과 더불어 허브솔트를 곁들여 취향에 따라 다양한 맛을 즐기실 수 있습니다. 편의시설 및 서비스3 예약 단체 이용 가능 무선 인터넷 결제수단1 제로페이 SNS1 블로그 https://blog.naver.com/cadfor
성북구 [SEP] 정릉1동 [SEP] 서울 성북구 솔샘로6길 28-15 1층 [SEP] 전여친전남친 [SEP] None [SEP] None [SEP] https://naver.me/54klqleK [SEP] 가격도 좋고 양도 많고 맛있네요. 비 오는 날이면 자주 올 것 같아요. 친절해요 오늘은 3명이서 와서 고기전에 부찌 두개다 시켰어요ㅋㅋ키키 셋이서 배터지게 먹었슴당😆 화요일날 방문했는데도 사람이 꽉 차있었어요! 역시 맛있고 화장실도 깨끗하고 최고👍🏻 굴전 푸짐하고 맛있어요. 부추하고 싸먹으니 더 맛있네요. 기분좋게 먹고 갑니다. 전 맛있어요 [SEP] 소개 맛있는 전, 찌개 파는 우리동네맛집!! 편의시설 및 서비스2 포장 예약 SNS1 홈페이지 https://www.make

## RAG
- MVP
  - llm: `gpt-3.5-turbo`
  - chain: `RetrievalQA`
  - Index: `FAISS`

In [20]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo")

In [21]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=faiss_retriever
)

In [23]:
generated_plan = qa_chain.invoke("성북구에서 할 수 있는 식도락 여행 코스 짜줘")

In [27]:
print(generated_plan['result'])

저는 식도락 여행 코스를 짜는 데 도움을 줄 수 있습니다. 성북구에서 할 수 있는 식도락 여행 코스로는 다음과 같은 장소들을 추천할 수 있습니다:

1. 열린마당
   - 위치: 서울 성북구 삼선동 동소문로2길 27
   - 메뉴: 안주가 푸짐하고 맛있는 맛집으로 소개되어 있습니다.
   - 특이사항: 가성비가 좋아서 자주 찾는 곳이라고 합니다.

2. 동망봉 금문장 중국집
   - 위치: 서울 성북구 보문동 보문동6가 209-280
   - 메뉴: 중국집으로 유명한 곳입니다.
   - 특이사항: 동망봉을 넘어 숭인근린동까지의 코스에 위치해 있습니다.

3. 성곽마을 여행자센터
   - 위치: 서울 성북구 삼선동 삼선교로4길 89
   - 특이사항: 성곽투어를 할 때 들르기 좋은 카페로 소개되어 있습니다.

4. 옛고을장충왕족발
   - 위치: 서울 성북구 장위1동 월계로36길 27 꿈의숲대명루첸아파트
   - 메뉴: 가마솥조림 기법으로 만든 쫄깃한 족발과 부드러운 보쌈이 인기 있는 곳입니다.

5. 플랜비
   - 위치: 서울 성북구 보문동 고려대로1길 25 1층
   - 특이사항: 맥주 안주로 감튀 갈릭이 맛있다고 소문난 카페입니다.

이러한 장소들을 방문하면 성북구에서 다채로운 식도락 여행을 즐길 수 있을 것입니다.
